In [11]:
!pip install POT

In [12]:
import numpy as np
import numpy.random as random

import matplotlib.pyplot as plt
import ot

In [13]:
def ndot(a,b):
    return a[0]*b[0]-a[1]*b[1]

In [14]:
def clamp(value, min_val, max_val):
    return max(min_val, min(value, max_val))

In [15]:
def sdf_triangle(p,r=0.5):
    k=np.sqrt(3.0)
    p[0]=np.abs(p[0])-r
    p[1]=p[1]+r/k
    idx = (p[:,0]+k*p[:,1]>0.0)
    if(idx>0.0):
       p[idx,0] = (p[idx,0]-k*p[idx,1])/2.0
       p[idx,1] = (-k*p[idx,0]-p[idx,1])/2.0
    p[0]-=clamp(p[0],-2.0*r,0.0);
    max_p = np.maximum(p, 0.0)
    length_p = np.sqrt(np.sum(max_p**2, 1))
    return -np.linalg.norm(p)*np.sign(p[1])

In [16]:
def sdf_pentagon(p,r=0.5):
  k=np.array[0.809016994,0.587785252,0.726542528];
  p.x=np.abs(p[0])
  l=(-k[0],k[1])
  m=(k[0],k[1])
  p-=2.0*min(np.dot(l,p),0.0)*l;
  p-=2.0*min(np.dot(m,p),0.0)*m
  p-=(clamp(p[0],-r*k[2],r*k[2]),r);
  max_p = np.maximum(p, 0.0)
  length_p = np.sqrt(np.sum(max_p**2, 1))
  return -np.linalg.norm(p)*math.sin(p[1])

In [17]:
def gen_grid(resolution, low=-1.0, high=1.0):
    idx = np.linspace(low, high, num=resolution)
    x, y = np.meshgrid(idx, idx)
    V = np.concatenate((x.reshape((-1,1)), y.reshape((-1,1))), 1)

    return V

In [18]:
def sample_and_normalize(f, grid, grid_size):
    '''
    Sample f on the grid and normalize it.
    Assume f>0 outside and <0 inside.
    '''
    fv = f(grid)

    # >0 inside
    fv = -fv

    # f is the characteristic function for {f>0}
    fv[fv>=0.0] = 1.0
    fv[fv<0.0] = 0.0

    # normalize
    fv = fv / np.sum(fv)

    # reshape to have the same shape as grid
    fv = fv.reshape(grid_size, grid_size)

    return fv

In [19]:
grid_size = 32
grid = gen_grid(grid_size)

# f1 and f2 are prob. distribution corresponding to f1 and f2
f1 = sample_and_normalize(sdf_triangle, grid, grid_size)
f2 = sample_and_normalize(sdf_pentagon, grid, grid_size)

A = np.array([f1,f2])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
nb_images = 5
reg = 0.004

v1 = np.array((1, 0))
v2 = np.array((0, 1))

fig, axes = plt.subplots(1, nb_images, figsize=(7, 7))
plt.suptitle("Convolutional Wasserstein Barycenters in POT")
cm = "Blues"

for i in range(nb_images):
    tx = float(i) / (nb_images - 1)

    weights = (1 - tx) * v1 + tx * v2

    if i == 0:
        axes[i].imshow(f1, cmap=cm)
    elif i == (nb_images - 1):
        axes[i].imshow(f2, cmap=cm)
    else:
        # call to barycenter computation
        axes[i].imshow(
            ot.bregman.convolutional_barycenter2d(A, reg, weights), cmap=cm
        )
    axes[i].axis("off")

plt.tight_layout()
plt.show()